In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 긍*부정 분류

In [5]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멋쟁이사자차럼/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
train = pd.read_csv(f"{DATA_PATH}review_train.csv")
test = pd.read_csv(f"{DATA_PATH}review_test.csv")

train.shape, test.shape
train
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2000 non-null   object
 1   review  2000 non-null   object
 2   target  2000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 47.0+ KB


In [ ]:
%pip install kiwipiepy
from kiwipiepy import Kiwi
text = train["review"][0]
text

In [8]:
kiwi = Kiwi()
result = kiwi.tokenize(train["review"])
train_list = [ [ t.form for t in tokens ] for tokens in result ]
result = kiwi.tokenize(test["review"])
test_list = [ [ t.form for t in tokens ] for tokens in result ]
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(max_features=500)
train_data = vec.fit_transform(
    [ " ".join(t) for t in train_list ]
).A
test_data = vec.transform(
    [ " ".join(t) for t in test_list ]
).A
train_data.shape, test_data.shape
(train_data.sum(axis=1) == 0).sum()
target = train["target"].to_numpy().reshape(-1, 1)
target.shape

(2000, 1)

## 데이터셋

In [ ]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, x, y=None):
        self.x , self.y = x, y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, i):
        item = {}
        item["x"] = torch.Tensor(self.x[i])
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[i])
        return item
dt = ReviewDataset(train_data,target)
dl= torch.utils.data.DataLoader(dt, batch_size=2)
batch = next(iter(dl))
batch

## 모델 클래스

In [10]:
class ResidualBlock(torch.nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.fx = torch.nn.Sequential(
            torch.nn.Linear(in_features, in_features),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(in_features, in_features)
        )
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        fx = self.fx(x)
        hx = fx + x
        return self.relu(hx)
class Net(torch.nn.Module):
    def __init__(self, in_features, n_layers=8):
        super().__init__()

        self.init_layer = torch.nn.Sequential(
            torch.nn.Linear(in_features, in_features // 2),
            torch.nn.BatchNorm1d(in_features // 2),
            torch.nn.LeakyReLU()
        )
        res_list = [ ResidualBlock(in_features//2) for _ in range(n_layers) ]
        self.seq = torch.nn.Sequential(*res_list)
        self.output_layer = torch.nn.Linear(in_features//2, 1)

    def forward(self, x):
        x = self.init_layer(x)
        x = self.seq(x)
        return self.output_layer(x)
Net(train_data.shape[1])(batch["x"])

tensor([[-0.5295],
        [-0.6667]], grad_fn=<AddmmBackward0>)

## 학습 루프

In [11]:
def train_loop(dl, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in dl:
        pred = model(batch["x"].to(device))
        loss = loss_fn(pred, batch["y"].to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dl)
    return epoch_loss

## 테스트 루프

In [12]:
@torch.no_grad()
def test_loop(dl, model, loss_fn, device):
    epoch_loss = 0
    model.eval()

    act = torch.nn.Sigmoid()
    pred_list = []
    for batch in dl:
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn(pred, batch["y"].to(device) )
            epoch_loss += loss.item()

        pred = act(pred)
        pred = pred.to("cpu").numpy()
        pred_list.append(pred)

    pred = np.concatenate(pred_list)
    epoch_loss /= len(dl)
    return epoch_loss, pred

## 학습하기

In [14]:
n_splits = 5
batch_size = 32
epochs = 100
loss_fn = torch.nn.BCEWithLogitsLoss()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
cv = KFold(n_splits, shuffle=True, random_state=SEED)
is_holdout = False
reset_seeds(SEED)
score_list = []
for i, (tri, vai) in enumerate(cv.split(train_data)):
    # 학습 데이터
    train_dt = ReviewDataset(train_data[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    # 검증 데이터
    valid_dt = ReviewDataset(train_data[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체 및 옵티마이저 생성
    model = Net(train_data.shape[1]).to(device)
    optimizer = torch.optim.Adam( model.parameters() )

    patience = 0 # 조기 종료 조건을 주기 위한 변수
    best_score = 0 # 현재 최고점수
    for _ in tqdm(range(epochs)):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)
        pred = np.where(pred > 0.5, 1, 0)
        score = accuracy_score(target[vai], pred)
        print(train_loss, valid_loss, score)

        patience += 1
        if score > best_score:
            best_score = score
            patience = 0
            torch.save( model.state_dict(), f"model{i}.pt" )

        if patience == 10:
            break

    score_list.append(best_score)
    print(f"ACC 최고점수: {best_score}")

    if is_holdout:
        break
print(score_list)
print(np.mean(score_list))

  0%|          | 0/100 [00:00<?, ?it/s]

0.6260003209114074 0.5925454405637888 0.695
0.379820748269558 0.6105662721853989 0.7125
0.26754079788923263 0.6897672002132123 0.7375
0.1563432063162327 0.8922322564400159 0.715
0.13917258907109498 1.0078993554298694 0.7275
0.14890590753406285 0.815742436509866 0.7025
0.156663689725101 0.9710836593921368 0.7025
0.1479032376408577 1.3063740844909961 0.7
0.17703445203602314 0.682049444088569 0.6925
0.1635565023124218 0.9010361387179449 0.685
0.12212781194597483 1.0967345879628108 0.69
0.10248114230344071 1.3402236172786126 0.7025
0.12383770775049925 1.172918134010755 0.6925
ACC 최고점수: 0.7375


  0%|          | 0/100 [00:00<?, ?it/s]

0.628304991722107 0.5954232445129981 0.7525
0.3878269359469414 0.5100278453185008 0.74
0.29675365768373013 0.6154235097078177 0.7525
0.22123784929513932 0.7782755700441507 0.7425
0.216577732488513 1.1273493354137127 0.7275
0.16998404145240784 0.9165945442823263 0.7225
0.14857698656618595 1.1132087363646581 0.7325
0.1672697953879833 0.9669479750669919 0.7275
0.12981294900178908 0.9978178418599642 0.71
0.13078484300523996 0.8576521437901717 0.735
0.13966008685529233 1.2096684208283057 0.7225
ACC 최고점수: 0.7525


  0%|          | 0/100 [00:00<?, ?it/s]

0.6642198544740677 0.5948674495403583 0.7475
0.4184854620695114 0.561614488179867 0.72
0.2798328714072704 0.6347491557781513 0.745
0.19114400312304497 0.8473711862013891 0.745
0.2078106202930212 0.8849363900147952 0.7225
0.14876782663166524 0.9523641432707126 0.7325
0.17062129363417625 1.0509078594354482 0.745
0.13576409161090852 1.1649943360915551 0.7425
0.16012893240898848 0.8926766721101907 0.7425
0.12906271897256374 1.287862733579599 0.7375
0.1517935087531805 1.0358810837452228 0.75
0.13139153217896818 0.8038533708223929 0.745
0.1751857240498066 1.047303855419159 0.72
0.14816871456801892 1.1960475811591516 0.735
0.139954680390656 1.2749497111027057 0.7375
0.13069343231618405 1.379403327520077 0.725
0.11586355868726969 1.3721713515428395 0.7425
0.11306424800306558 1.3565188073194945 0.7375
0.1095341221243143 1.3353846004376044 0.72
0.11848981844261289 1.4961209274255312 0.7375
0.12889645154820756 1.73997108065165 0.7275
ACC 최고점수: 0.75


  0%|          | 0/100 [00:00<?, ?it/s]

0.6230823534727097 0.5793119439711938 0.7325
0.3807615339756012 0.5555983942288619 0.7225
0.2577189654111862 0.6313256323337555 0.725
0.18841964587569238 0.7338531429951007 0.7175
0.168511633425951 0.7756723165512085 0.6875
0.15561677128076554 0.8259520484850957 0.705
0.13779956076294184 1.1796976923942566 0.695
0.14126431873068213 0.9004086508200719 0.69
0.14739614255726338 1.0362345392887409 0.6975
0.15704521991312503 1.0173881604121282 0.71
0.104984563998878 0.9668565942690923 0.705
ACC 최고점수: 0.7325


  0%|          | 0/100 [00:00<?, ?it/s]

0.6312319874763489 0.5774480196145865 0.73
0.40092132151126864 0.53205629495474 0.725
0.24804230347275735 0.6905173796873826 0.73
0.21048673287034034 0.7514290970105392 0.7275
0.1744207288324833 0.778675187092561 0.7275
0.19104268342256547 1.4477126323259795 0.72
0.1541249580681324 1.1337340336579542 0.715
0.1373986041545868 1.5294681856265435 0.7275
0.14331772286444902 1.7673752216192393 0.725
0.1400418544560671 1.1257527608137865 0.755
0.13947141848504543 1.404688468346229 0.745
0.19706335794180632 0.8127730488777161 0.7225
0.11796038679778575 1.8310037897183344 0.7325
0.16969131004065274 1.1795270580511827 0.7375
0.10699086855631322 1.3543335657853346 0.7275
0.11324104625731707 1.5654681554207435 0.7125
0.10806102165952325 2.0845795961526723 0.7025
0.10741430275142193 1.8120651566065276 0.7075
0.1343352037668228 1.2887794879766612 0.725
0.10168681403622032 1.5512618743456328 0.7175
ACC 최고점수: 0.755
[0.7375, 0.7525, 0.75, 0.7325, 0.755]
0.7455


## 예측하기

In [ ]:
test_dt = ReviewDataset(test_data)
test_dl = torch.utils.data.DataLoader(test_dt, shuffle=False, batch_size=batch_size)
pred_list = []
for i in range(n_splits):
    model = Net(train_data.shape[1]).to(device)
    state_dict = torch.load(f"model{i}.pt", weights_only=True)
    model.load_state_dict(state_dict)

    _, pred = test_loop(test_dl, model, None, device)
    pred_list.append(pred)
pred = np.mean(pred_list, axis=0)
pred = np.where(pred>0.5, 1, 0)
pred.shape

# 맞춤법 교정한 파일 가져오기

In [15]:
# 3. 엑셀 파일에서 데이터 읽기
file_path = '/content/drive/MyDrive/멋쟁이사자차럼/data/신흥시장_맞춤법교정완료파일.xlsx'
df = pd.read_excel(file_path)

In [16]:
len(df)

8728

In [17]:
df

,source,id,review,date,revisit,source_file
0,eave,부니86,분위기도 너무 좋고 따뜻한 공간이에요 😚 재방문 의사 100%입니다.,12.17.화,1번째 방문,eave_2024-12-23_14-14-12
1,eave,정열75,카페 분위기도 좋고 커피도 맛있습니다!.,12.16.월,1번째 방문,eave_2024-12-23_14-14-12
2,eave,guri093099,넘 맛있어요 >_<.,12.14.토,1번째 방문,eave_2024-12-23_14-14-12
3,eave,dld****,해방촌 eave.,12.14.토,1번째 방문,eave_2024-12-23_14-14-12
4,eave,토리629,매장도 너무 깔끔하고 친절하시고 맛있어요 🤍🤍.,12.13.금,1번째 방문,eave_2024-12-23_14-14-12
...,...,...,...,...,...,...
8723,흑해,메리0,여기 짱 맛있어요! 저희가 밀크티랑 화연만 화학 연구원이 만드는 에이드를 시켰는데 ...,23.7.1.토,1번째 방문,흑해_2024-12-23_14-24-02
8724,흑해,생각씨,날씨 더웠는데 해방촌 올라오자마자 흑해 카페에서 화연만 에이드 마셨어요! 에이드 줜...,23.7.1.토,1번째 방문,흑해_2024-12-23_14-24-02
8725,흑해,Skun21,터키식 커피? 홍차? 메네멘? 카이막까지 맛볼 수 있는 흑해 🤩 저번에 터키 커피랑...,23.7.1.토,1번째 방문,흑해_2024-12-23_14-24-02
8726,흑해,AD ASTRA57,멜랑꼴리한 날 편하게 이색적인 안주랑 너무 무겁지 않은 세 미스 위트 와인이랑 혼술...,23.6.29.목,1번째 방문,흑해_2024-12-23_14-24-02


In [19]:
df['clean_review']= df['review'].str.replace("[^\w ]+", "", regex=True).str.lower()

In [22]:
filtered_data =df[df['clean_review'].str.len() >= 3]  # 길이 3미만 리뷰 제거

- 한글 리뷰만 남기고 다 제거해보기

In [25]:
import re  # re 모듈 임포트

In [23]:
def extract_word(text):
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', text)
    return result

In [27]:
df['clean_review'] = df['clean_review'].apply(lambda x:extract_word(x))

## 키워드 추출

In [40]:
%pip install konlpy
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 41.3 MB/s eta 0:00:00


In [41]:
okt = Okt()
words = " ".join(df['clean_review'].tolist())
words = okt.morphs(words,stem=True)

In [43]:
len(words)

121036

In [42]:
# 한글자 제거
remove_one_word = [x for x in words if len(x)>1 ]
len(remove_one_word)

84354

In [44]:
# 단어별 빈도를 확인
from collections import Counter
frequent = Counter(remove_one_word).most_common()

In [45]:
frequent

[('좋다', 4203),
 ('맛있다', 3772),
 ('하다', 2672),
 ('너무', 2455),
 ('있다', 1637),
 ('커피', 1452),
 ('카페', 1193),
 ('먹다', 1189),
 ('오다', 1053),
 ('분위기', 1011),
 ('친절하다', 969),
 ('보다', 810),
 ('가다', 748),
 ('이다', 747),
 ('해방촌', 725),
 ('맛집', 686),
 ('디저트', 669),
 ('사장', 643),
 ('같다', 579),
 ('음료', 579),
 ('넘다', 576),
 ('예쁘다', 568),
 ('진짜', 526),
 ('방문', 490),
 ('에서', 485),
 ('루프', 464),
 ('케이크', 463),
 ('자다', 458),
 ('싶다', 449),
 ('정말', 444),
 ('귀엽다', 366),
 ('이에요', 357),
 ('인테리어', 356),
 ('많다', 351),
 ('라테', 345),
 ('으로', 335),
 ('하고', 325),
 ('않다', 304),
 ('없다', 300),
 ('이쁘다', 290),
 ('들다', 288),
 ('자리', 265),
 ('공간', 264),
 ('되다', 260),
 ('아늑하다', 259),
 ('최고', 256),
 ('예요', 250),
 ('남산', 250),
 ('추천', 245),
 ('다음', 244),
 ('매장', 243),
 ('시간', 239),
 ('인데', 233),
 ('나오다', 230),
 ('사진', 224),
 ('메뉴', 221),
 ('사람', 220),
 ('수플레', 219),
 ('이라', 215),
 ('음식', 215),
 ('직원', 215),
 ('크림', 211),
 ('친구', 206),
 ('가격', 206),
 ('여기', 204),
 ('까지', 192),
 ('많이', 191),
 ('조용하다', 189),
 ('주다', 186),
 ('들어

- 불용어 좀 더 상의해서 제거해야할듯.
- 일단은 러프하게 그냥 하자.

In [51]:
with open('/content/drive/MyDrive/멋쟁이사자차럼/data/stopwords.txt', 'r', encoding = 'CP949') as f:
    list_file = f.read()

# 불용어 리스트 생성
stopwords = list_file.split(",")  # 쉼표로 나누기

# 불용어 제거
remove_stopwords = [x for x in remove_one_word if x not in stopwords]
len(remove_stopwords)

60499


In [52]:
Counter(remove_stopwords).most_common()

[('좋다', 4203),
 ('맛있다', 3772),
 ('커피', 1452),
 ('카페', 1193),
 ('먹다', 1189),
 ('분위기', 1011),
 ('친절하다', 969),
 ('해방촌', 725),
 ('맛집', 686),
 ('디저트', 669),
 ('사장', 643),
 ('음료', 579),
 ('예쁘다', 568),
 ('방문', 490),
 ('루프', 464),
 ('케이크', 463),
 ('자다', 458),
 ('귀엽다', 366),
 ('인테리어', 356),
 ('많다', 351),
 ('라테', 345),
 ('이쁘다', 290),
 ('자리', 265),
 ('공간', 264),
 ('아늑하다', 259),
 ('최고', 256),
 ('남산', 250),
 ('추천', 245),
 ('매장', 243),
 ('사진', 224),
 ('메뉴', 221),
 ('사람', 220),
 ('수플레', 219),
 ('음식', 215),
 ('직원', 215),
 ('크림', 211),
 ('친구', 206),
 ('가격', 206),
 ('조용하다', 189),
 ('들어가다', 181),
 ('크루', 180),
 ('좋아하다', 176),
 ('자주', 165),
 ('가게', 163),
 ('느낌', 162),
 ('주문', 160),
 ('생각', 158),
 ('기분', 152),
 ('아쉽다', 152),
 ('멋지다', 150),
 ('찾다', 148),
 ('날씨', 148),
 ('앉다', 144),
 ('카이', 143),
 ('처음', 141),
 ('갈다', 131),
 ('분들', 130),
 ('말다', 129),
 ('다양하다', 128),
 ('비싸다', 128),
 ('아주', 127),
 ('계단', 125),
 ('넓다', 124),
 ('깔끔하다', 122),
 ('쿠키', 122),
 ('편하다', 121),
 ('마시다', 120),
 ('시원하다', 120),
 ('만들다', 1

- 빈도수 적은 단어들 보기

In [53]:
for item, count in Counter(remove_stopwords).most_common():
    if count==1:
        print(item)

리스
행운
안정감
마일
드하
오래전
벼루
앤북앤
필름
비스킷
테이
시끌시끌하다
다저트
흡족하다
꾸역꾸역
우디
구원
카패
내미다
플루트
그램
겉바속쫀
잃어버리다
라디오
나시
타트
휘낭세
쏟아지다
순백
책방
코스터
쫀낭시에
늦잠
자고
쇼핑몰
세미나
차라
손해
계열
적재적소
초르르
가구
마주
과자류
푸릇
입지
신규
테이크
수운
도중
삼초
직장인
파격
곳임
빗방울
마루
에게도
식히다
시달리다
타지
취미
아티스트
괜스레
외식
축복
밤바람
마침표
혜택
항목
견줄
로우
레코드
턴테이블
청음
갑작스레
빌리다
과제
사근사근하다
사운드
한층
신선
올드
덕덕구스
심심하다
내기
오버
비행기
블라인드
음주
썸네일
경차
시원시원하다
빠작하
일욜
후끈하다
초대형
불꽃놀이
열악하다
접다
숙이다
구치소
철창
할머니
밥상
견주들
소개팅
종일
타운
숭실고
요거트
통틀어
아른거리다
진솔하다
쌩맥
이마트
붉다
물들다
간맥
비프
퍼먹는
주간
불고기
볗로입니
페퍼로니
볼률
조정
놀이터
하늘색
바라보이다
명동
익스트림
복잡복잡
생맥주
날리다
부서지다
엘리베이터
긴가민가하다
멀티
벚꽃
뒹굴거릴
마땅찮다
쿠션
쌀쌀
난방
홍대
팟카파오므쌉
피나콜라다
취합니
치앙마이
탱글탱글
면은
꼬릿파오무쌉
다져지다
양념
해외
모반
월길
얌커무양
말리부
파인애플
복판
우이
태원
와글와글
하하호호
찰깍
뿌팟퐁
커리
퓨전
푸팟퐁
아시안
항정살
구이
차만
입술
착색
만큼은
크래커
어린이날
허다
똠얌
파라다이스
카리브해
모히또
짜조
화이트와인
홀려
드감
마심
단술
말로
짱짱맨
탄자니아
활짝
단물
교양
쌓다
느긋하다
약제
생김새
갈라
키위
윙스
헬초코
이를
피로
파리
임보
연락
굿잡
공수
토모
찌롤
늘어지다
커퓌
임명
속세
훈훈하다
노원
강동
땡큐
둘째
서럽다
기운
우리나라
최곤
녹찻물
이냐
뽀짝
꾸덕스
친화력
부럽다
축하
강강추
큐트
사무실
어랏
멜번
금손이셔
덩이
코리
백이십
뭉치다
필리핀
녹아내리다
귀염뽀짝
긍정
얻다
왕강추
유제품
큐키
문구
더부룩하다
대추야자
박해
삐약이
뀰잼
장문
하얗다
구멍
뻘쭘했어
놨다
핵맛
쓱싹대다


- 3회 이하로 나온 단어들 일단 삭제!! .. 이것도 상의해봐야 할듯하다.

In [54]:
minimum_count = 3
more_than_one_time= []
for i in tqdm(range(len(remove_stopwords))):
    tmp = remove_stopwords[i]
    if remove_stopwords.count(tmp) >= minimum_count:
        more_than_one_time.append(tmp)

100%|██████████| 60499/60499 [01:33<00:00, 649.53it/s]


In [67]:
len(more_than_one_time)

57069

In [79]:
text = set(more_than_one_time)

In [82]:
# CountVectorizer 객체 생성 (필요에 따라 추가 파라미터 조정 가능)
vectorizer = CountVectorizer()

# 리뷰 데이터를 CountVectorizer로 변환
data_features = vectorizer.fit_transform(df['clean_review'].tolist())

In [85]:
# TF-IDF 변환기 생성 및 데이터 변환
tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(data_features)

# 단어 인덱스와 단어를 매핑하는 사전 생성
vocab = {v: k for k, v in vectorizer.vocabulary_.items()}

# 출력 확인
print("단어와 인덱스 매핑:", vocab)
print("TF-IDF Matrix Shape:", tf_idf_vect.shape)

단어와 인덱스 매핑: {6418: '분위기도', 2217: '너무', 11719: '좋고', 3701: '따뜻한', 1045: '공간이에요', 11208: '재방문', 10390: '의사', 10819: '입니다', 13259: '카페', 13400: '커피도', 4644: '맛있습니다', 4650: '맛있어요', 14971: '해방촌', 4807: '매장도', 1680: '깔끔하고', 13171: '친절하시고', 6412: '분위기가', 11737: '좋아요', 13630: '크리스마스라', 14036: '트리랑', 4124: '리스들로', 1793: '꾸며져', 10849: '있고', 1931: '나무로', 13907: '테이블이', 11639: '조화로워요', 11014: '자주', 9560: '오는', 3250: '동네', 13316: '카페예요', 13391: '커피', 4617: '맛있고', 8480: '아늑합니다', 11850: '주로', 15201: '혼자', 9875: '와서', 9158: '업무하는데요', 14360: '평일은', 11607: '조용하고', 11864: '주말은', 6912: '사람이', 4413: '많아요', 1476: '근처에', 6268: '볼일이', 10884: '있어', 10063: '우연치', 8814: '않게', 3471: '들렀는데', 5299: '무드', 11804: '좋은', 13291: '카페를', 12642: '찾아서', 11259: '저는', 15095: '행운아입니다', 9592: '오래오래', 5986: '번창하세요', 13260: '카페가', 8472: '아늑하고', 11614: '조용해서', 13406: '커피랑', 3637: '디저트도', 4626: '맛있네요', 6437: '분위기의', 12666: '찾으시다면', 12960: '추천드립니다', 13392: '커피가', 10506: '이뻐요', 8483: '아늑해요', 15184: '호지티', 11754: '좋아하는데', 3946: '라테로',

- 본격 키워드 추출
- 이 사이에 긍부정으로 나누는 과정 있으면 좋을 듯 하다.

- 그 다음에 긍정, 부정 별로 키워드... 왜 이 걸 선택했는지... 카페별로! 있으면 좋을듯.
- 나는 지금 전체에 대해서 했지만, 카페별로 해야될듯 하다 ^.^
- 즉 하... 아냐 걍 여기서 멈춰..................